In [1]:
import tensorflow as tf

In [2]:
D = 8

In [3]:
with tf.name_scope("constant"):
    t = tf.placeholder(tf.float64, shape=())
    H = tf.reshape(tf.constant([[0.25,0,0,0],[0,-0.25,0.5,0],[0,0.5,-0.25,0],[0,0,0,0.25]],dtype=tf.float64),[2,2,2,2],name="Hamiltonian")
    I = tf.reshape(tf.constant([[1.,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]],dtype=tf.float64),[2,2,2,2],name="Identity")
    expH = I + t * H

In [4]:
with tf.name_scope("variables"):
    A = tf.Variable(tf.random_uniform([D, D, 2], minval=-1., maxval=1.,dtype=tf.float64),name="A")
    B = tf.Variable(tf.random_uniform([D, D, 2], minval=-1., maxval=1.,dtype=tf.float64),name="B")
    EAB = tf.Variable(tf.ones(D,dtype=tf.float64), name="EAB")
    EBA = tf.Variable(tf.ones(D,dtype=tf.float64), name="EBA")
    L = tf.Variable(tf.random_uniform([D, D], minval=-1., maxval=1.,dtype=tf.float64),name="L")
    R = tf.Variable(tf.random_uniform([D, D], minval=-1., maxval=1.,dtype=tf.float64),name="R")
    
    sA = tf.Variable(A, name="sA")
    sB = tf.Variable(B, name="sB")
    sEAB = tf.Variable(EAB, name="sEAB")
    sEBA = tf.Variable(EBA, name="sEBA")
    sL = tf.Variable(L, name="sL")
    sR = tf.Variable(R, name="sR")
    
    init = tf.global_variables_initializer()

In [5]:
with tf.name_scope("parameter"):
    REAB = tf.reciprocal(EAB,name="REAB")
    REBA = tf.reciprocal(EBA,name="REBA")
    
    EA = tf.multiply(tf.multiply(A,tf.reshape(EBA,[D,1,1])),tf.reshape(EAB,[1,D,1]),name="EA")
    EB = tf.multiply(tf.multiply(B,tf.reshape(EAB,[D,1,1])),tf.reshape(EBA,[1,D,1]),name="EB")
    
    AB = tf.tensordot(EA,EB,[[1],[0]],name="AB")
    BA = tf.tensordot(EB,EA,[[1],[0]],name="BA")
    
    HAB = tf.tensordot(AB,expH,[[1,3],[0,1]],name="HAB")
    HBA = tf.tensordot(BA,expH,[[1,3],[0,1]],name="HBA")
    
    LA = tf.tensordot(tf.tensordot(L,EA,[[1],[0]]),EA,[[0,2],[0,2]],name="LA")
    LAB = tf.tensordot(tf.tensordot(LA,EB,[[1],[0]]),EB,[[0,2],[0,2]],name="LAB")
    
    BR = tf.tensordot(tf.tensordot(R,EB,[[1],[1]]),EB,[[0,2],[1,2]],name="BR")
    ABR = tf.tensordot(tf.tensordot(BR,EA,[[1],[1]]),EA,[[0,2],[1,2]],name="ABR")

In [6]:
with tf.name_scope("updateAB"):
    S, U, V = tf.svd(tf.reshape(tf.transpose(HAB,[0,2,1,3]),[2*D,2*D]))
    ABnEAB = tf.sqrt(S[:D],name="nE")
    ABnA = tf.transpose(tf.multiply(tf.reshape(U[:,:D],[D,2,D]),tf.reshape(REBA,[D,1,1])),[0,2,1],name="nA")
    ABnB = tf.transpose(tf.multiply(tf.reshape(V[:,:D],[D,2,D]),tf.reshape(REBA,[D,1,1])),[2,0,1],name="nB")
    SupdateAB = tf.assign(sEAB,ABnEAB/tf.norm(ABnEAB)),tf.assign(sA,ABnA/tf.norm(ABnA)),tf.assign(sB,ABnB/tf.norm(ABnB))
    with tf.control_dependencies(SupdateAB):
        updateAB = tf.assign(A,sA),tf.assign(B,sB),tf.assign(EAB,sEAB)

with tf.name_scope("updateBA"):
    S, U, V = tf.svd(tf.reshape(tf.transpose(HBA,[0,2,1,3]),[2*D,2*D]))
    BAnEBA = tf.sqrt(S[:D],name="nE")
    BAnB = tf.transpose(tf.multiply(tf.reshape(U[:,:D],[D,2,D]),tf.reshape(REAB,[D,1,1])),[0,2,1],name="nB")
    BAnA = tf.transpose(tf.multiply(tf.reshape(V[:,:D],[D,2,D]),tf.reshape(REAB,[D,1,1])),[2,0,1],name="nA")
    SupdateBA = tf.assign(sEBA,BAnEBA/tf.norm(BAnEBA)),tf.assign(sB,BAnB/tf.norm(BAnB)),tf.assign(sA,BAnA/tf.norm(BAnA))
    with tf.control_dependencies(SupdateBA):
        updateBA = tf.assign(B,sB),tf.assign(A,sA),tf.assign(EBA,sEBA)

In [7]:
with tf.name_scope("updateL"):
    SupdateL = tf.assign(sL,LAB/tf.norm(LAB))
    with tf.control_dependencies((SupdateL,)):
        updateL = tf.assign(L,sL)
        
with tf.name_scope("updateR"):
    SupdateR = tf.assign(sR,ABR/tf.norm(ABR))
    with tf.control_dependencies((SupdateR,)):
        updateR = tf.assign(R,sR)

In [8]:
with tf.name_scope("Energy"):
    LA = tf.tensordot(L,A,[[0],[0]])
    LAE = tf.multiply(LA,tf.reshape(EAB*EAB,[1,D,1]))
    LAEB = tf.tensordot(LAE,B,[[1],[0]])
    LAEBR = tf.tensordot(LAEB,R,[[2],[0]])
    LAEBRA = tf.tensordot(LAEBR,A,[[0],[0]])
    LAEBRAE = tf.multiply(LAEBRA,tf.reshape(EAB*EAB,[1,1,1,D,1]))
    LAEBRAEB = tf.tensordot(LAEBRAE,B,[[3,2],[0,1]])
    E_AB = tf.tensordot(LAEBRAEB,H,[[0,1,2,3],[0,1,2,3]])/tf.tensordot(LAEBRAEB,I,[[0,1,2,3],[0,1,2,3]])

    LAEBE = tf.multiply(LAEB,tf.reshape(EBA*EBA,[1,1,D,1]))
    LAEBEA = tf.tensordot(LAEBE,A,[[2],[0]])
    LAEBEAE = tf.multiply(LAEBEA,tf.reshape(EAB*EAB,[1,1,1,D,1]))
    LAEBEAEA = tf.tensordot(LAEBEAE,A,[[0,1],[0,2]])
    LAEBEAEAE = tf.multiply(LAEBEAEA,tf.reshape(EAB*EAB,[1,1,1,D]))
    LAEBEAEAEB = tf.tensordot(LAEBEAEAE,B,[[3],[0]])
    LAEBEAEAEBE = tf.multiply(LAEBEAEAEB,tf.reshape(EBA*EBA,[1,1,1,D,1]))
    LAEBEAEAEBEA = tf.tensordot(LAEBEAEAEBE,A,[[3],[0]])
    LAEBEAEAEBEAE = tf.multiply(LAEBEAEAEBEA,tf.reshape(EAB*EAB,[1,1,1,1,D,1]))
    LAEBEAEAEBEAEB = tf.tensordot(LAEBEAEAEBEAE,B,[[4],[0]])
    LAEBEAEAEBEAEBB = tf.tensordot(LAEBEAEAEBEAEB,B,[[1,6],[0,2]])
    LAEBEAEAEBEAEBBR = tf.tensordot(LAEBEAEAEBEAEBB,R,[[5,4],[0,1]])
    E_BA = tf.tensordot(LAEBEAEAEBEAEBBR,H,[[0,1,2,3],[0,1,2,3]])/tf.tensordot(LAEBEAEAEBEAEBBR,I,[[0,1,2,3],[0,1,2,3]])

    E = (E_AB+E_BA)/2

    tf.summary.scalar('E', E)

In [ ]:
config = tf.ConfigProto()
#config.graph_options.optimizer_options.global_jit_level=tf.OptimizerOptions.ON_2
#config.log_device_placement=True
config.device_count["CPU"]=8
config.device_count["GPU"]=1

sess = tf.Session(config=config)
sess.run(init)
merged = tf.summary.merge_all()
logger = tf.summary.FileWriter('/tmp/data/%d'%D,sess.graph)
tt = 0

In [ ]:
ep = 0.1
for i in xrange(1):
    for j in xrange(1000):
        sess.run(updateAB,feed_dict={t: -4*ep})
        sess.run(updateBA,feed_dict={t: -4*ep})
        sess.run(updateBA,feed_dict={t: -4*ep})
        sess.run(updateAB,feed_dict={t: -4*ep})
        #for j in xrange(100):
        sess.run((updateL,updateR))
        logger.add_summary(sess.run(merged), tt)
        tt += 1

logger.flush()
print sess.run(E)